data: https://data.gov.uk/dataset/723c243d-2f1a-4d27-8b61-cdb93e5b10ff/uk-local-authority-and-regional-carbon-dioxide-emissions-national-statistics-2005-to-2019

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import esda
from esda import Moran
from libpysal import weights
from libpysal.weights.contiguity import Queen
from splot.esda import moran_scatterplot, lisa_cluster, plot_local_autocorrelation

from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes, mark_inset
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import matplotlib.colors as mcolors
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.font_manager as font_manager
import matplotlib.patches as mpatches


from esda.getisord import G_Local

%matplotlib inline

In [ ]:
df_emissions=pd.read_csv("data//2005-19_Local_Authority_CO2_emissions.csv")

In [ ]:
df_emissions['LA CO2 Sector'].unique() #Check the sectors available

In [ ]:
df_emissions['LA CO2 Sub-sector'].unique()

In [ ]:
df_public=df_emissions[(df_emissions['LA CO2 Sector']=='Public Sector') & (df_emissions['Calendar Year']==2019)] #only keep public sector

In [ ]:
#get total sum of public sector sub-sectors
df_public_grouped=df_public.groupby(['Local Authority','Local Authority Code'])['Territorial emissions (kt CO2)'].sum().reset_index()

In [ ]:
df_public_grouped=df_public_grouped.rename(columns={'Territorial emissions (kt CO2)':'co2_kt'})

In [ ]:
zipfile = "zip://data//Local_Authority_Districts__April_2019__UK_BFE_v2-shp.zip!Local_Authority_Districts__April_2019__UK_BFE_v2.shp"

In [ ]:
gdf = gpd.read_file(zipfile)

In [ ]:
la_co2=gdf.merge(df_public_grouped, left_on='LAD19CD',right_on='Local Authority Code',how='left')

In [ ]:
la_co2=la_co2.set_crs(27700,allow_override=True)


In [ ]:
cities=gpd.read_file("data/cities.geojson")
cities=gpd.GeoDataFrame(cities, geometry='geometry',crs=4326)
cities=cities.to_crs(27700)

In [ ]:
f,ax=plt.subplots(figsize=(22,22))

ax.set_axis_off()


pallete=['#E2D2FF','#E458A7','#7D2359','#E93630','#2F0202'] #custom cmap

cmap = LinearSegmentedColormap.from_list(
    'mycmap',pallete)

missing_kwds={
        "color": "lightgrey",
        "edgecolor": "blue",
        "hatch": "///",
        "label": "Missing values",
    }   #mapping LA's with no Valus

la_co2.plot(column='co2_kt', ax=ax, scheme='quantiles', k=5, legend=True,cmap=cmap,legend_kwds={'fontsize': 'xx-large'},
            missing_kwds=missing_kwds) #plot UK map

#add text to the legend
legend=ax.get_legend()
font = font_manager.FontProperties(weight='bold',
                                   style='normal', size=12)
legend.set_title('CO2 Emissions kt (5 quantiles)',prop=font)


#title
plt.title('Local Authority Public Sector C02 Emission (kt) 2019*', fontsize=32, weight='bold')

plt.text(1, -0.03,'*3 Local Authorities missing', ha='center', va='center', transform=ax.transAxes,fontsize=15,weight='bold')

#plot cities
cities.plot(ax=ax, color='white')

for x, y, label in zip(cities.geometry.x, cities.geometry.y, cities.city):
    ax.annotate(label, xy=(x, y), xytext=(3, 3), textcoords="offset points",fontsize=12,weight='bold',color='green')


#create inset

#get bounds (if you don't have numbers just get the bounds of a subset e.g. df[df['region']=='Foo'].total_bounds)
london_bounds=[503899.40652557, 157410.44000192, 569337.62687451, 205799.40644699]
# nw_bounds=[330811.64,370000,393867.85,430000]

#inset map and setting up axes -LDN


axins = zoomed_inset_axes(ax, 3, loc=4)
minx,miny,maxx,maxy =london_bounds

axins.set_xlim(minx, maxx)
axins.set_ylim(miny, maxy)

axins.tick_params(axis=u'both', which=u'both',length=0)

mark_inset(ax, axins, loc1=2, loc2=1, fc="none", ec="0.5")
plt.setp(axins.get_xticklabels(), visible=False)
plt.setp(axins.get_yticklabels(), visible=False)
la_co2.plot(column='co2_kt',ax=axins,scheme='quantiles',k=5, missing_kwds=missing_kwds, cmap=cmap,legend=False)

# cities.plot(ax=axins, color='white')


# #inset map and setting up axes -NW
# axins1 = zoomed_inset_axes(ax, 2, loc=2)

# minx1,miny1,maxx1,maxy1 =nw_bounds


# axins1.set_xlim(minx1, maxx1)

# axins1.set_ylim(miny1, maxy1)

# axins1.tick_params(axis=u'both', which=u'both',length=0)
# # cmap = LinearSegmentedColormap.from_list(
# #     'mycmap',pallete)
# mark_inset(ax, axins1, loc1=2, loc2=1, fc="none", ec="0.5")
# plt.setp(axins1.get_xticklabels(), visible=False)
# plt.setp(axins1.get_yticklabels(), visible=False)
# la_co2.plot(column='co2_kt',ax=axins1,scheme='quantiles',k=5, missing_kwds=missing_kwds, cmap=cmap,legend=False)

# cities.plot(ax=axins1, color='white')

# for x, y, label in zip(cities.geometry.x, cities.geometry.y, cities.city):
#     axins1.annotate(label, xy=(x, y), xytext=(-20, 3), textcoords="offset points",fontsize=12,weight='bold', rotation=40,
#                     color='blue')


plt.show()

## Cluster Analysis (Pysal)

In [ ]:
w=weights.Queen.from_dataframe(la_co2, idVariable='LAD19CD')

In [ ]:
gdf_noi=la_co2.set_index('LAD19CD').drop(w.islands).reset_index()

In [ ]:
w=weights.Queen.from_dataframe(gdf_noi, idVariable='LAD19CD')

In [ ]:
w.islands

In [ ]:
w.transform='R'

In [ ]:
gdf_noi['w_co2']=weights.lag_spatial(w,gdf_noi['co2_kt'])

In [ ]:
lisa=esda.Moran_Local(gdf_noi['co2_kt'],w)

In [ ]:
f,ax=plt.subplots(figsize=(30,30))
lisa_cluster(lisa,gdf_noi,ax=ax)

## Cluster Analysis g*-ord

In [ ]:
# def filter_out_london_lsoas(df):
#     """
#     function to filter out London LSOAs
#     """
    
#     searchfor = ['City of London', 'Barking and Dagenham', 'Barnet', 'Bexley'
#            , 'Brent 0', 'Bromley', 'Camden', 'Croydon', 'Ealing', 'Enfield',
#              'Greenwich', 'Hackney', 'Hammersmith and Fulham', 'Haringey'
#              , 'Harrow', 'Havering', 'Hillingdon', 'Hounslow', 'Islington',
#              'Kensington and Chelsea', 'Kingston upon Thames', 'Lambeth', 
#              'Lewisham', 'Merton', 'Newham', 'Redbridge', 'Richmond upon Thames', 
#              'Southwark', 'Sutton', 'Tower Hamlets', 'Waltham Forest', 'Wandsworth', 'Westminster']

#     #make df for just london
#     df= df[df['LSOA11NM'].str.contains('|'.join(searchfor))]
    
#     return df
def create_weights_and_gstar(df,col):
    
    if 'index' not in df.columns.tolist():  #need this for creating weight s
        df=df.reset_index()
    
    
    # Create the spatial weights matrix using queens neighbourhood
    w = weights.Queen.from_dataframe(df, idVariable='index')
    
    #drop all islands from dataframe
    df=df.drop(w.islands, axis=0)
    #need to recreate spatial weights matrix after removing islands
    w = weights.Queen.from_dataframe(df, idVariable='index')
    
    # Row standardize the matrix
    w.transform = 'R'
    
#     #spatial lag of column
#     df['w_{}'.format(col)] = weights.lag_spatial(w, df[col])
    
    g= esda.getisord.G_Local(df[f'{col}'], w, star=True)  
    
    df['sig']=g.p_sim < 0.05
    df['hh']=g.Zs > 0
    df['ll']=g.Zs < 0
    
    
    return df

def geostar_map(df, ax):
    '''
    Create a cluster map
    ...
    
    Arguments
    ---------
    g     :  G_Local Gi*
             Object from the computation of the G statistic
    geog   : GeoDataFrame
             Table aligned with values in `g` and containing 
             the geometries to plot
    ax     : AxesSubplot
             `matplotlib` axis to draw the map on

    Returns
    -------
    ax     : AxesSubplot
             Axis with the map drawn
    '''
    ec = '0.8'
    
    # Plot non-significant clusters
    ns = df.loc[df.sig==False, 'geometry']
    ns.plot(ax=ax, color='lightgrey', edgecolor=ec, linewidth=0.1, aspect=1)
    # Plot HH clusters
    hh = df.loc[(df.hh==True) & (df.sig==True), 'geometry']
    hh.plot(ax=ax, color='crimson', edgecolor=ec, linewidth=0.1,  aspect=1)
    # Plot LL clusters
    ll = df.loc[(df.ll) & (df.sig==True), 'geometry']
    ll.plot(ax=ax, color='royalblue', edgecolor=ec, linewidth=0.1,  aspect=1)
    
    return ax

def plot_g_map_with_inset(gdf,col):
    """
    plot spatial autocorrelation map with inset
    Params:
    gdf: geopandas dataframe
    col: str, column name where LISA is being explored
    """
    
    df_lisa=gdf.copy()  #create a copy df to plot the local indicator of spatial autocorrelation
    df_lisa=create_weights_and_gstar(df_lisa,col)
    

    f,ax=plt.subplots(figsize=(20,20))
    ax.set_axis_off()
    
    geostar_map(df_lisa, ax=ax)
    
#     df_lisa_london=filter_out_london_lsoas(df_lisa)
    
#     minx,miny,maxx,maxy =  df_lisa_london.total_bounds

    
#     axins = zoomed_inset_axes(ax, 3, loc=2)
#     axins.set_xlim(minx, maxx)
#     axins.set_ylim(miny, maxy)
#     axins.tick_params(axis=u'both', which=u'both',length=0)

#     plt.setp(axins.get_xticklabels(), visible=False)
#     plt.setp(axins.get_yticklabels(), visible=False)
    
#     geostar_map(df_lisa_london,ax=axins)
    
    ax.set_title('Local indicator of spatial autocorrelation using Gi* for {}'.format(col), size=15)
    
    #add legend
    red_patch = mpatches.Patch(color='crimson', label='Hotspot')
    blue_patch = mpatches.Patch(color='royalblue', label='Coldspot')
    grey_patch = mpatches.Patch(color='lightgrey', label='Not significant')



    ax.legend(loc=1, handles=[red_patch, blue_patch, grey_patch], fontsize= 14)

    
    return plt.show()
    #return plt.savefig("gi_star for {}.png".format(col))


In [ ]:
la_co2['co2_kt'].plot()

In [ ]:
plot_g_map_with_inset(la_co2,'co2_kt')

### Global Moran's I (test)

In [ ]:
def plot_global_morans_i(gdf, column):
    
    df=gdf.copy()
    
    if 'index' not in df.columns.tolist():  #need this for creating weight s
        df=df.reset_index()
    
    
    # Create the spatial weights matrix using queens neighbourhood
    w = weights.Queen.from_dataframe(df, idVariable='index')
    
    #drop all islands from dataframe
    df=df.drop(w.islands, axis=0)
    #need to recreate spatial weights matrix after removing islands
    w = weights.Queen.from_dataframe(df, idVariable='index')
    
    y = df[column].values

    moran = Moran(y, w)
    
    
    
    fig, ax = plt.subplots(figsize=(15,15))
    
    moran_scatterplot(moran, aspect_equal=True,ax=ax)
    ax.set_title("Global Moran Scatter plot for {} with Score:{} and significance:{}.".format(column, moran.I.round(2), moran.p_sim))
    
    return plt.show()